# 第二届·2022量子精选论文复现挑战赛 第 24 题
## 论文题目：
## Multiqubit state learning with entangling quantum generative adversarial networks
## 论文链接：https://journals.aps.org/pra/abstract/10.1103/PhysRevA.106.032429
### 复现人：chzh32

### 论文简介：
经典的生成对抗网络（generative adversarial network，GAN）模型已被广泛应用于图像生成、改善视频游戏分辨率、暗物质建模等。
GAN 的核心原理是使用两个网络共同完成一个对抗游戏。其中一个网络作为生成器 generator，其生成虚假的数据来“以假乱真”欺骗另一个作为鉴别器的网络。而鉴别器则努力区别数据到底是真实的，还是生成器生成的。但由于梯度消失和模式崩溃以及收敛问题，训练 GAN 是很困难的。
由经典 GAN 发展出来的量子版本被称为 QGAN，被用于生成 CNOT 门、近似纯态和 MINST 数据集等任务。但起初的 QGAN 由于纳什均衡的非唯一性，并不能保证一直收敛。

在量子信息中，经常需要比较两个未知量子态是否等效，这可以通过交换测试来做到。为了节省测试所需资源，有文章提出破坏性交换测试方案。破坏交换测试实验可以根据测量结果的统计数据来判断两个输入的量子态是否等价。

但在面临噪声情况下，标准的交换测试可能并不能提供足够的效果。本文使用量子生成对抗模型 QGAN 在噪声环境下训练拟设结构来作为交换测试线路。该方案相比标准线路在噪声下具有更好的效果。

本项目即对标准交换测试线路和 QGAN 生成的测试线路有无噪声的情况下效果进行比较。

# 复现流程及结果

下面我们考虑文章中双量子比特情况。根据双量子比特破坏交换测试方案，测试结果由测量结果决定，如果测量结果为 0111、1011、1101 或 1110 则测试失败，表明输入的两个双量子比特态不等效；否则等效。

标准的破坏交换测试线路为：

<img src='./src/2_qubit_swap_test_circ.png' width='70%'>

其运行流程为：

1. 固定用于生成目标量子态的真实量子线路 real_circ 的参数，训练拟设 ansatz_circ 线路参数，使的测量结果为 0111、1011、1101 和 1110 的概率最小。当得到上述四种测量结果的概率和为 0 时，ansatz_circ 生成的量子态等效于 real_circ 生成的量子态。

2. 比较上一步获得的拟设线路 ansatz_circ 的参数与真实线路 real_circ 的参数，并比较两线路所得量子态之间的保真度。

下图为 QGAN 的线路图，主要包含三部分：用于生成目标量子态的真实线路 real_circ、用于模仿真实线路的生成器线路 gen_circ 和起到交换测试线路效果的鉴别器线路 disc_circ。 

<img src='./src/2_qubit_qgan_circ.png' width='90%'>

QGAN 的运行流程为：
1. 先在 real_circ 和 gen_circ 输入相同的参数，保证输出量子态等效。训练鉴别器 disc_circ 的参数，使的测量结果为 0111、1011、1101 和 1110 的概率最小。当得到上述四种测量结果的概率和为 0 时，该鉴别器线路 dics_circ 等效于交换测试线路。

2. 将上一步获得的鉴别器线路 disc_circ 的参数作为鉴别器的固定参数。以鉴别器作为交换测试线路，训练生成器线路 gen_circ 使得测量结果为  0111、1011、1101 和 1110 的概率最小。当得到上述四种测量结果的概率和为 0 时，gen_circ 生成的量子态等效于 real_circ 生成的量子态。

3. 比较上一步获得的生成器参数与真实线路参数，并比较两线路所得量子态之间的保真度。

**注**：为模拟噪声对线路的影响，我们对线路中每个量子门后面都施加一个振幅阻尼信道，其取值为 0.01。

通过运行 `main.ipynb` 下的 `主程序 0` 可以获得标准破坏交换测试方案在噪声下的表现；运行 `主程序 1` 可在噪声环境下对 QGAN 的鉴别器进行训练；运行 `主程序 2` 可在噪声环境下对 QGAN 的生成器进行训练。

以下为具体的复现结果：

|               |            | Fidelity | $\theta_0$ |  $\theta_1$  |  $\theta_2$  | $\theta_3$  | $\theta_4$  | $\theta_5$  |
|   ----        | ----       | ----     |    ----    |      ----    |      ----    |      ----   |      ----   |      ----   |
|真实数据       | $\theta^*$ | 1        | 0.035      |  2.861       |  0.606       |  0.361      |  6.174      |4.513        |
|标准交换测试数据| $\theta_p$ | 0.999984  | -0.21003175| -0.04623173  |0.77686948   |2.5473932     |3.20918606  |  1.13139806  |
|对抗交换测试数据| $\theta_a$ | 0.999978   |-0.65835516|  3.15744377  |-0.07791132  | 3.02851366   |3.48606626  |  1.61923996  |

## 自验环境

硬件：Windows 10 系统、 4U8G 256G 存储 

软件：Mindspore 1.6.0、Mindquantum 0.8.0